# Crytocurrency Data Exercise

The first section of this notebook contains the script used to ingest data from the CoinMarketCap API and load it into BigQuery. Each step labeled with markdown text. The second section contains the SQL queries used to answer the questions in the exercise

## I - CoinMarketCap Data Pipeline

In [63]:
import json, pandas as pd, requests, logging
from pandas.io.json import json_normalize
from google.datalab import Context
import google.datalab.bigquery as bq
import google.datalab.storage as storage
  
# global variable names
bucket_name = 'coin-market-cap-bucket'
csv_file_name = 'cmc_data.csv'
log_file_name = 'coinmarketcap.log'

# begin logging
logging.basicConfig(filename=log_file_name, format='%(asctime)s - %(message)s')  
logging.info('############### SCRIPT START ###############')

# CoinMarketCap API call function
def api_call(querystring, uri, payload, api_key):
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Content-Type': "application/json"
    }
    try:
        response = requests.request("GET", uri, data=payload, headers=headers, params=querystring)
        return response
    except requests.exceptions.RequestException as e:
        print(e)

#### 1. Pull current data for all cryptocurrencies using the CoinMarketCap API

In [64]:
# storing credentials in gcs bucket to avoid hardcoding
# in production environment, this file would be stored in a non-public bucket only accessible throught the service
# account used by this script. For purposes of this exercise, file is in the same public bucket as the output file
%gcs read --object gs://coin-market-cap-bucket/config/config.json --variable config

config = json.loads(config.decode('utf-8'))
cmc_api_uri = config['api']['coinMarketCap']['uri']
cmc_api_key = config['api']['coinMarketCap']['key']

querystring = {"limit":"5000"}
payload = "{\n\tstart: 1,\n\tconvert: 'USD'\n}"

# call api fetch function which returns the data and any response codes for error handling
response = api_call(querystring, cmc_api_uri, payload, cmc_api_key)

logging.info('Step 1 - Pull CoinMarketCap API data - Complete')

#### 2. Save this data as a CSV file

In [65]:
# adhering to instructions to save data as .csv file. Depending on size, the data could be streamed directly to bucket
# to avoid saving it locally first

if not response.status_code == 200:
  error_msg = json.loads(response.text)['status']['error_message']
  print('Error fetching CoinMarketCap API data. {}'.format(error_msg))
  logging.error('Error fetching CoinMarketCap API data. %s', error_msg)
else:
  # convert response string to json. Cryptocurrency data is in the "data" key of the response json
  crypto_data_list = json.loads(response.text)['data']

  # flatten json structure into pandas dataframe with rows and columns
  crypto_data_tbl_df = json_normalize(crypto_data_list)

  # ensure column names don't have periods (.)
  crypto_data_tbl_df.columns = [x.strip().replace('.', '_') for x in crypto_data_tbl_df.columns]

  # remove any NaN values from dataframe
  crypto_data_tbl_df.fillna(value='', inplace=True)

  # save local csv file
  crypto_data_tbl_df.to_csv('/tmp/'+ csv_file_name, index=False)

  # save crypto_data_tbl_df schema to use later when creating table in BigQuery
  coin_data_schema = bq.Schema.from_data(crypto_data_tbl_df)
  
  logging.info('Step 2 - Save CSV File - Complete')

#### 3. Upload the CSV to a Google Cloud Storage Bucket

In [66]:
object_folder = 'cryptocurrency-data'
bucket_key = object_folder + '/' + csv_file_name
local_path = '/tmp/' + csv_file_name
bucket_path = 'gs://' + bucket_name + '/' + bucket_key

# using gsutil command line tool to move local file to GCS
!gsutil cp  $local_path  $bucket_path

# verify file was successfully uploaded to GCS
bucket_object = storage.Object(bucket_name, bucket_key)
if not bucket_object.exists():
  print('File not successfully uploaded to bucket. Issue logged')
  logging.error('%s was not uploaded to bucket successfully.', csv_file_name)
  exit(1)
else:  
  logging.info('Step 3 - Upload to GCS Bucket - Complete')

Copying file:///tmp/cmc_data.csv [Content-Type=text/csv]...
/ [1 files][539.3 KiB/539.3 KiB]                                                
Operation completed over 1 objects/539.3 KiB.                                    


#### 4. Move cryptocurrency data from GCS bucket to BigQuery

In [67]:
if response.status_code == 200:
  # Create the dataset if it doesn't exist. If it exists, it will not be overwritten
  bq.Dataset('cryptocurrency').create()

  # Create coin data table based on previously stored dataframe schema if it doesn't exist. Table WILL be overwritten
  coin_data_table = bq.Table('cryptocurrency.coin_data').create(schema = coin_data_schema, overwrite = True)

  # insert coin data
  coin_data_table.load(bucket_path, mode='append',
                    source_format = 'csv', csv_options=bq.CSVOptions(skip_leading_rows = 1))

logging.info('Step 4 - Insert into BigQuery Table - Complete')
logging.info('############### SCRIPT END ###############')

In [68]:
# Transfer log file to GCS
object_folder = 'config'
bucket_key = object_folder + '/' + log_file_name
bucket_path = 'gs://' + bucket_name + '/' + bucket_key

# using gsutil command line tool to move local file to GCS
!gsutil cp  $log_file_name  $bucket_path

# cleanup in the next section

Copying file://coinmarketcap.log [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


In [82]:
%%bash
rm -f $local_path
rm -f $log_file_name

## II - Exercise Queries

#### 1. How many coins have a USD price greater than $8,000?

In [70]:
%%bq query -n coins_gt_8k
select count(name) as num_coins_gt_8k from cryptocurrency.coin_data
where quote_usd_price > 8000

In [71]:
%bq execute -q coins_gt_8k

num_coins_gt_8k
2


#### 2. What is the total market cap of the top 100 cryptocurrencies (in USD)?

In [72]:
%%bq query -n top_100
select sum(z.quote_USD_market_cap) as total_market_cap
from (
select quote_USD_market_cap from cryptocurrency.coin_data
order by cmc_rank
limit 100
) z

In [73]:
%bq execute -q top_100

total_market_cap
129775946451.8717


#### 3. Which coins have an available supply less than $5M?

In [74]:
%%bq query -n avail_supply_lt_5M
select name from cryptocurrency.coin_data
where cast(total_supply as FLOAT64) < 5000000
order by total_supply

In [75]:
%bq execute -q avail_supply_lt_5M

name
Project-X
ThoreCoin
Maker
Kolion
Ethereum Lite
Mixin
KUN
Byteball Bytes
Dalecoin
Melon


#### 4. Which 5 coins have seen the greatest percentage growth in the last week?

In [76]:
%%bq query -n highest_growth_week
select name from cryptocurrency.coin_data
order by quote_USD_percent_change_7d desc
limit 5

In [77]:
%bq execute -q highest_growth_week

name
PlayerCoin
Fujinto
Privatix
Earth Token
ZCore


#### 5. How many ticker symbols contain the letter "X" ?

In [78]:
%%bq query -n x_in_symbol
select count(*) as num_symbol_w_X from cryptocurrency.coin_data
where upper(symbol) like '%X%'

In [79]:
%bq execute -q x_in_symbol

num_symbol_w_X
276
